<a href="https://colab.research.google.com/github/harmeetkaur2711/level-up-sql-3211372/blob/main/SDA_Harmeet_Dhanyee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment for Senior Data Analyst Role**

In [1]:
# Mounting google drive to access file from the drive directly

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Installing all the necessary libraries and importing them**

In [ ]:
%%time

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
# install Basemap
!apt-get install libgeos-3.5.0
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
# Installing Chart_Studio
!pip install chart_studio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-gdal is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gdal-bin

E: Package 'python-gdal' has no installation candidate
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6 python3-rtree
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 365 kB of archives.
After this operation, 1,799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive

In [ ]:
# importing libraries

import numpy as np
import pandas as pd
import glob
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
import plotly_express as px
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.basemap import Basemap
import chart_studio.plotly as py
import plotly.graph_objs as go #importing graphical objects
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

## **Importing the Parking Tags Data and combining them into one dataframe**

Please note that I have taken data for last 3 years only, so that the system does not get sluggish, but if required the same analysis can be performed on any year of data by loading appropriate files

In [ ]:
path = r'/content/drive/My Drive/SDA' # use your path
all_files = glob.glob(path + "/Parking*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0)
    li.append(frame)

df = pd.concat(li, axis=0, ignore_index=True)

### **Viewing Parking Data and Performing Initial Analysis**

In [ ]:
#Viewing data
df.head(10)

In [ ]:
df.info()

### Performing Data Cleaning for Parking Infraction data

Lets first view province data to check if we have data for only one province or more provinces in Cananda

In [ ]:
df.province.value_counts().head(20)

In [ ]:
# Viewing percentage of rows that contain data of Ontario
df.province[df.province=='ON'].value_counts()/df.province.count()

From above analysis we can see that the licesnce plate of the cars majorly belonged to Ontario however around 4% of them are from different states of Canada and US

#### Checking Location Data

From the table we can see that Location2 caontains the main street address where the infraction has happened, and location3 denotes how location 2 is related to location 4.  

In [ ]:
# Checking values of location3 to see if they are consistent
df.location3.value_counts().head(20)

In [ ]:
# Replacing differently spelled codes with one code
pd.options.mode.chained_assignment = None  # default='warn'
df.location3.replace(dict.fromkeys(['WO','W OF','W/OF','WEST OF','WEST/OF','W/S','W O','WS'], 'W/O'), inplace=True)
df.location3.replace(dict.fromkeys(['EO','E OF','E/OF','EAST OF','EAST/OF','E/S','E O','ES'], 'E/O'), inplace=True)
df.location3.replace(dict.fromkeys(['NO','N OF','N/OF','NORTH OF','NORTH/OF','N/S','N O','NS'], 'N/O'), inplace=True)
df.location3.replace(dict.fromkeys(['SO','S OF','S/OF','SOUTH OF','SOUTH/OF','S/S','S O','SS'], 'S/O'), inplace=True)
df.location3.replace(dict.fromkeys(['AT','ON','OF','NEAR','NR','ACROSS'], 'AND'), inplace=True)
df.location3.value_counts().head(20)

In [ ]:
# Replacing all non-consistent data with blank
X = ['W/O','E/O','N/O','S/O','AND']
df.loc[~df.location3.isin(X), 'location3'] = ''
df.location3.value_counts()

In [ ]:
# Replacing NaN values with blanks for Location4 as well
df.location4.replace(np.nan,'',inplace=True)

#### Converting date of infraction from integer value to proper datetime format

In [ ]:
df['date_of_infraction']=pd.to_datetime(df.date_of_infraction, format='%Y%m%d')

In [ ]:
df.head()

## Viewing Green Park Data

This is a Json file that contains information of all Green Parking locations around Toronto.

In [ ]:
# importing file
import json
with open('/content/drive/MyDrive/SDA/green-p-parking-2019.json') as f:
  temp_view = json.load(f)


In [ ]:
# Viewing file
print(temp_view)

In [ ]:
# Viewing data for one Green Park to see what kind of information is available
print(json.dumps(temp_view['carparks'][1], indent=4))

### Performing Data Cleaning for Green Park Data

Now that we know what kind of information is available in the Json we will try to retrive this information into a pandas Dataframe

In [ ]:
# Reading the json file and converting it into pandas Dataframe
green_park = pd.read_json('/content/drive/MyDrive/SDA/green-p-parking-2019.json')
green_park

In [ ]:
# Retrieving Necessary information into columns of Pandas Dataframe
green_park['id'] = green_park.apply(lambda x: json.dumps(x['carparks']['id']).strip('\"'), axis=1)
green_park['lat'] = green_park.apply(lambda x: json.dumps(x['carparks']['lat']).strip('\"'), axis=1)
green_park['lng'] = green_park.apply(lambda x: json.dumps(x['carparks']['lng']).strip('\"'), axis=1)
green_park['address'] = green_park.apply(lambda x: json.dumps(x['carparks']['address']).strip('\"'), axis=1)

In [ ]:
#Viewing data
green_park

## Viewing TTC locations data

We dont need much data cleaning as we mostly require only latitude and longitude data

In [ ]:
#Reading required file
ttc = pd.read_csv('/content/drive/My Drive/SDA/stops.txt')

In [ ]:
# Viewing Data
ttc.head()

## Performing Data Analysis and Finding answers to the query

Now that our data is in better shape we can find what are the top 20 ticket infractions frequency-wise and revenue-wise.



### 3.1.1 Top 20 ticket infractions - Frequency-wise

In [ ]:
#Top 20 ticket infractions - Frequency-wise
df.infraction_code.value_counts().head(20)

### 3.1.2 Top 20 ticket infractions - Revenue-wise

In [ ]:
# Top 20 ticket infractions - Revenue-wise

table = pd.pivot_table(df, values='set_fine_amount', index=['infraction_code'], aggfunc=np.sum)
table.rename(columns={'set_fine_amount':'revenue'}, inplace=True)
table.sort_values(by='revenue', ascending=False).head(20)

### 3.1.3 Total Revenue Generated from all the tickets

In [ ]:
# Calculationg the total revenue generated from all the tickets
print('Total Revenue Generated =',table.revenue.sum())

### 3.1.4a Distance to closest parking lots for top 20 infractions

#### Finding top 20 Infraction locations

Creating a new column 'Location' with combination of location2 and location4 of Parking Infractions data

In [ ]:
# Defining function for combining location 2 and location 4

# Please note that we have already cleaned data for location3 so that we can use that information to combine location2 and location4
def full_loc(loc2, loc3, loc4):
  loc=''
  if loc3 == "":                        #If no data is available in location3
    loc = str(loc2)                     #then location will be equal to location 2
  else:                                 #otherwise
    loc = str(loc2) + ', ' + str(loc4)  #location will be combination of location 2 and location 4
  return loc + ', Toronto, ON'



In [ ]:
# applying function full_loc to Parking Infraction dataframe
# This cell will take some time to run
df['location']=df.apply(lambda x: full_loc(x['location2'], x['location3'], x['location4']), axis=1)

In [ ]:
# Viewing data
df.head()

In [ ]:
# Finding top 20 infraction locations
x = df.location.value_counts().head(20)
top_20_loc = x.to_frame(name='value_counts')
top_20_loc.reset_index(inplace=True)
top_20_loc.rename(columns={'index':'infraction_location'}, inplace=True)
top_20_loc

#### Finding Latitude and Longitude of top 20 infraction location

In [ ]:
# Using Geocode to find latitude and longitude of a descriptive location
locator = Nominatim(user_agent="myGeocoder")

def find_loc(loc):
  location = locator.geocode(loc)
  if location:
    return (location.latitude, location.longitude)
  else:
    return (np.nan, np.nan)


In [ ]:
# Applying function find_loc to our data
top_20_loc['lats']=top_20_loc.apply(lambda row: find_loc(row['infraction_location']), axis=1)
top_20_loc[['latitude','longitude']] = pd.DataFrame(top_20_loc['lats'].tolist(), index=top_20_loc.index)


In [ ]:
# Now we have the latitude and longitude information which can further be used to find distance from nearest Green Park Location
top_20_loc

In [ ]:
# Writing a function to find nearest parking location
from geopy.distance import geodesic

def find_nearest_parking(infraction_location, list_of_carparks):
  dist = list_of_carparks.apply(lambda x: geodesic(infraction_location, (x['lat'], x['lng'])).m, axis=1) #Creating list of distance of our point from all parking lots
  nearest_parking_dist = dist.min()                                                                      # Finding parking lot with minimum distance
  nearest_parking_address = list_of_carparks.iloc[dist.idxmin()]['address']                              # Finding address of parking lot with minimum distance
  return nearest_parking_dist, nearest_parking_address



In [ ]:
# Applying function find_nearest_parking to our dataset
nearest_parking = top_20_loc.apply(lambda x: find_nearest_parking(x['lats'], green_park), axis = 1)
top_20_loc[['nearest_parking_dist','nearest_parking_address']] = pd.DataFrame(nearest_parking.tolist(), index=top_20_loc.index)


#### Table containing information of distance of closest parking lots from top 20 infraction locations

In [ ]:
top_20_loc

### 3.1.4b Distance of Closest TTC stop for top 20 infraction locations

In [ ]:
# Creating finction to find nearest TTC stop
def find_nearest_ttc(infraction_location, list_of_ttc):
  dist = list_of_ttc.apply(lambda x: geodesic(infraction_location, (x['stop_lat'], x['stop_lon'])).m, axis=1)
  nearest_ttc_dist = dist.min()
  nearest_ttc_address = list_of_ttc.iloc[dist.idxmin()]['stop_name']
  return nearest_ttc_dist, nearest_ttc_address

In [ ]:
#Applying function find_nearest_ttc
nearest_ttc = top_20_loc.apply(lambda x: find_nearest_ttc(x['lats'], ttc), axis = 1)
top_20_loc[['nearest_ttc_dist','nearest_ttc']] = pd.DataFrame(nearest_ttc.tolist(), index=top_20_loc.index)
top_20_loc

So Finally we have our data of top 20 infraction locations with nearest park location and nearest TTC

### 3.1.5a Impact of day of week on all infractions

In [ ]:
#First we are grouping data by day of week and infraction code to get desired result
dow = df['set_fine_amount'].groupby([df.infraction_code, df.date_of_infraction.dt.weekday]).count()

In [ ]:
# The result comes out as a multi index Series. Converting that multiindex Series to Datframe
dow_impact = dow.unstack(level=1,fill_value=0)

In [ ]:
# Renaming columns to make data more readable
dow_impact.rename(columns={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'},inplace=True)
dow_impact

from this table we can find frequency of a particular infraction code for any day of the week

**If we want to visualize the impact of week on any infraction code we can do so by plotting for particular infraction code as below. Here we can even show multiple plots if required**

In [ ]:
# By changing the value of n we can see impact of weekday for any infraction code. Here n denotes infraction code
n = 5
dow_impact.loc[n].plot.bar()

### **3.1.5b Impact of month on all infractions**

Similar to week we can analyze our data to see impact of month on frequency and type of infractions

In [ ]:
#grouping data by infraction code and month
mnth = df['set_fine_amount'].groupby([df.infraction_code, df.date_of_infraction.dt.month]).count()
mnth_impact = mnth.unstack(level=1,fill_value=0)

In [ ]:
# Renaming columns to make our data more readable
mnth_impact.rename(columns={1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'},inplace=True)
mnth_impact

In [ ]:
# By changing the value of n we can see impact of month for any infraction code. Here n denotes infraction code
n = 5   #Change value of n to look for a particular infraction code
mnth_impact.loc[n].plot.bar()

### 3.3.1.a.i Distributions of infractions by year

In [ ]:
# Grouping data by year and infraction code
yr = df['set_fine_amount'].groupby([df.infraction_code, df.date_of_infraction.dt.year]).count()

In [ ]:
yr_impact = yr.unstack(level=1,fill_value=0)
yr_impact

In [ ]:
yr_impact['total']=yr_impact[2016] + yr_impact[2017] + yr_impact[2018]
yr_impact.sort_values(by=['total'], ascending=False, inplace=True)

Plotting data for only top 20 infraction codes as plotting for all infraction codes will make the visualization messy

In [ ]:
yr_20 = yr_impact.head(20)
yr_20.drop(columns=['total'], inplace=True)
yr_impact_20 = yr_20.T

In [ ]:
# Viewing data yearwise
fig, ax=plt.subplots(3)

plt.rcParams["figure.figsize"] = (20,10)
fig.suptitle('Year Comparison', fontsize=20)
yr_20[2016].plot.bar(ax=ax[0], color='r')
yr_20[2017].plot.bar(ax=ax[1], color='g')
yr_20[2018].plot.bar(ax=ax[2], color='b')

ax[0].set_title('2016')
ax[1].set_title('2017')
ax[2].set_title('2018')

In [ ]:
# plotting Distribution of infractions by year
fig = plt.figure()
fig.suptitle('Year Impact', fontsize=20)
yr_impact_20.plot.bar(subplots=True, layout=(4,5))

### 3.3.1.a.ii  Distribution of Infractions by Month

Similar to year, we can visualize the same data month-wise

In [ ]:
mnth_impact['total'] = mnth_impact['Jan'] + mnth_impact['Feb'] + mnth_impact['Mar'] + mnth_impact['Apr'] + mnth_impact['May'] + mnth_impact['Jun'] + mnth_impact['Jul'] + mnth_impact['Aug'] + mnth_impact['Sep'] + mnth_impact['Oct'] + mnth_impact['Nov'] + mnth_impact['Dec']

In [ ]:
mnth_impact.sort_values(by=['total'], ascending=False, inplace=True)

In [ ]:
mnth_impact_20 = mnth_impact.head(20)
mnth_impact_20.drop(columns=['total'], inplace=True)
m_temp = mnth_impact_20.T

In [ ]:
m_temp.plot.bar(subplots=True, layout=(4,5))

### 3.3.1.b Distribution of top 20 infractions by fines

In [ ]:
# Grouping data by infraction code and finding sum of fines
fines = df['set_fine_amount'].groupby(df.infraction_code).sum()

In [ ]:
fines.head()

In [ ]:
fines.sort_values(ascending=False, inplace=True)

In [ ]:
#Creating subset of top 20 infractions
fines_20 = fines.head(20)

In [ ]:
# Plotting our data
fines_20.plot.bar()

### 3.3.2a Geographic distribution of top 20 infractions

Sorry but I have not worked much with geospatial data before so I was unable to do this in given time frame.

In [ ]:
import requests

# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "5e7a8234-f805-43ac-820f-03d7c360b588"}
package = requests.get(url, params = params).json()
print(package["result"])


for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df_geo = pd.DataFrame(data["result"]["records"])
        break
df_geo.head()

In [ ]:
xy = top_20_loc.longitude.mean()
xy

In [ ]:
fig1 = plt.figure(figsize=(20,10))

map = Basemap(projection='aeqd',
              lon_0 = -79.40,
              lat_0 = 43.68,
              width = 1000000,
              height = 1000000)

map.drawmapboundary(fill_color='aqua')
#Fill the continents with the land color
map.fillcontinents(color='coral',lake_color='aqua')
map.drawcoastlines()
plt.show()

In [ ]:
data = go.Scattergeo(lon = top_20_loc['longitude'], lat = top_20_loc['latitude'], mode='markers', marker = dict(symbol = 'star',size=5,colorscale = 'Reds'))
layout = dict(title = 'Green Park Locations')

In [ ]:
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)

In [ ]:
city_wards = pd.read_csv('/content/drive/My Drive/SDA/City Wards Data.csv')

In [ ]:
city_wards.head()

In [ ]:
# city_wards.geometry.plot(figsize=(40,20))
# plt.show